In [31]:
pip install -U langchain-community


Note: you may need to restart the kernel to use updated packages.


The following code conducts research on a topic and the search queries are provided by the default planner of deep agent. Research is a classic example of benefitting from multiple parallel subagent runs to research on multiple sub topics before coming together to converge and collate. The model used here is Azure GPT5 and a filesystem backend has been added.

In [33]:
from tavily import TavilyClient
from deepagents import create_deep_agent
import os
from typing import Literal
from langchain_mistralai import ChatMistralAI
from deepagents.backends import FilesystemBackend
from langchain_community.tools.file_management import (
    WriteFileTool,
    ReadFileTool,
    ListDirectoryTool
)



TAVILY_API_KEY="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"

In [34]:
tavily_client = TavilyClient(TAVILY_API_KEY)


In [35]:
def internet_search(query: str, 
            topic: Literal["general", "news", "finance"] = "general",
            max_results: int = 5,
            include_raw_content:bool = False):
    """Run a web search"""  
    response = tavily_client.search(query,max_results=max_results,
            include_raw_content=include_raw_content,
            topic=topic) 
    return response


In [40]:

instructions = """You are an expert researcher. Your job is to do a thorough research and publish a report. 
A tool has been provided to you to search the internet and get the information
## 'internet_search'
You can use this tool to run an internet search for a given query. You can specify
the maximum number of results, topic of the search and whether the raw 
search content should be included in the results.

You MUST:
1. Research the topic using the internet_search tool
2. Create a detailed written report
3. Save the final report as a markdown file using the filesystem backend
4. Name the file based on the topic

Rules:
- Use internet_search for research
- Write notes to notes.md
- Write final report to report.md using WriteFileTool


Always persist your findings to disk.
"""

In [37]:
MISTRALAI_API_KEY ="XN1gUFU0bOnw0nDGERkN37MAOHwc4WdQ"
if not os.getenv("MISTRALAI_API_KEY"):
    os.environ["MISTRALAI_API_KEY"] = MISTRALAI_API_KEY

In [38]:

model = ChatMistralAI(
    model="mistral-large-latest",
    api_key= MISTRALAI_API_KEY,
    temperature=0,
    max_retries=2,
    max_tokens=2000
    # other params...
) 

In [44]:
from pathlib import Path


path = Path("/Users/fatima/Desktop/vs code/storage")
path.mkdir(parents=True, exist_ok=True)

write_file = WriteFileTool(root_dir=str(path))
read_file = ReadFileTool(root_dir=str(path))
list_files = ListDirectoryTool(root_dir=str(path))

deep_agent = create_deep_agent(backend=FilesystemBackend(root_dir=str(path)),
                               model=model,
                               tools = [internet_search, write_file, read_file, list_files],
                    system_prompt=instructions)  

# response = deep_agent.invoke({"messages": [{"role": "user", "content": "What are all the latest additions to Langchain"}]})
response = deep_agent.invoke({
    "messages": [
        {
            "role": "user",
            "content": (
                "Research the latest additions to LangChain. "
                "After using tools, summarize the findings and "
                "WRITE a final report to report.md using WriteFileTool."
            )
        }
    ]
})

print(response)


/Users/fatima/Desktop/vs code/storage
{'messages': [HumanMessage(content='Research the latest additions to LangChain. After using tools, summarize the findings and WRITE a final report to report.md using WriteFileTool.', additional_kwargs={}, response_metadata={}, id='44fc93e1-06dc-4e0d-bf0d-b4247176aab4'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'gBgWsWpnm', 'function': {'name': 'internet_search', 'arguments': '{"query": "latest additions to LangChain 2024", "max_results": 5, "include_raw_content": true}'}, 'index': 0}, {'id': 'xS83eZKoX', 'function': {'name': 'internet_search', 'arguments': '{"query": "LangChain new features and updates June 2024", "max_results": 5, "include_raw_content": true}'}, 'index': 1}, {'id': '5zVs0u5jy', 'function': {'name': 'internet_search', 'arguments': '{"query": "LangChain documentation latest releases", "max_results": 5, "include_raw_content": true}'}, 'index': 2}, {'id': 'XwBgxJ3oj', 'function': {'name': 'internet_search', 'argu